## IMPORT LIBRARIES

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # Library to scrape information from web pages
import requests # library to handle requests

import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

## SCRAPE TABLE AND IGNORE CELLS WITH BOROUGH NOT ASSIGNED

In [2]:
origin = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

soup = BeautifulSoup(origin, 'lxml')

table = soup.find("table")
table_col = table.tbody.find_all("tr")

result = []
for tr in table_col:
    td = tr.find_all("td")
    col = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is "Not assigned\n".
    if col != [] and col[1] != "Not assigned\n":
                # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned\n" in col[2]: 
            col[2] = col[1]
        result.append(col)


### DATAFRAME WITH 3 COLUMNS AND REMOVE "\n" FROM TABLE

In [3]:
        
# Dataframe with 3 columns
df = pd.DataFrame(result, columns = ["PostalCode", "Borough", "Neighborhood"])

df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")

df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### GROUP NEIGHBORHOODS WITH SAME POSTAL CODE

In [4]:
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
print("Shape: ", df.shape)

Shape:  (103, 3)


### READ CSV FILE

In [6]:
df_coor = pd.read_csv("https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv")
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### MERGE BOTH DATA FRAMES

In [7]:
df_Toronto = pd.merge(df, df_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


### DROP POSTAL CODE COLUMN

In [8]:
df_Toronto.drop('Postal Code', axis=1, inplace=True)
df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### GET TORONTO CITY COORDINATES

In [9]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Toronto City Coordinates are {}, {}.'.format(latitude, longitude))


The Toronto City Coordinates are 43.6534817, -79.3839347.


### CREATE THE MAP OF TORONTO CITY AND ADD MARKERS

In [10]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    

    map_Toronto

I will work only with Boroughs that contain the word Toronto

In [11]:
# "ECDW" = [E]ast Toronto, [C]entral Toronto, [D]owntown Toronto, [W]est Toronto
df_Toronto_ECDW = df_Toronto[df_Toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_Toronto_ECDW.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Plot for this new Dataframe

In [12]:
map_Toronto_ECDW = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_Toronto_ECDW['Latitude'], df_Toronto_ECDW['Longitude'], df_Toronto_ECDW['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto_ECDW)  
    
map_Toronto_ECDW

### Define Foursquare Credentials and Version

In [123]:
CLIENT_ID = '4AXG5FDA2GOEVAOGJFFTJMP2MQMRG3CANKTQ2QFEM0B0IQ5O' # your Foursquare ID
CLIENT_SECRET = 'QDMDQLSFVX1QQ3BWVWICXB1NSJKA0M2XCCDA2LCGCKMI0XVJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### The first neighborhood in data frame "df_Toronto"

In [124]:
df_Toronto_ECDW.loc[0, 'Neighborhood']

'The Beaches'

Neighborhood's latitude and longitude values

In [125]:
neighborhood_latitude = df_Toronto_ECDW.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Toronto_ECDW.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_Toronto_ECDW.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Let's get the top 100 venues that are in The Beaches within a radius of 500 meters

In [126]:
search_query = 'The Beaches'
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, 
    neighborhood_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=4AXG5FDA2GOEVAOGJFFTJMP2MQMRG3CANKTQ2QFEM0B0IQ5O&client_secret=QDMDQLSFVX1QQ3BWVWICXB1NSJKA0M2XCCDA2LCGCKMI0XVJ&ll=43.67635739999999,-79.2930312&v=20180605&query=The Beaches&radius=500&limit=100'

In [127]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f08f1dda10064683ee05526'},
 'response': {'venues': [{'id': '4f430a70e4b0c5334e289db2',
    'name': 'Best Bathtub in the Beaches',
    'location': {'lat': 43.674591064453125,
     'lng': -79.2936019897461,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.674591064453125,
       'lng': -79.2936019897461}],
     'distance': 201,
     'cc': 'CA',
     'country': 'Canada',
     'formattedAddress': ['Canada']},
    'categories': [{'id': '4bf58dd8d48988d1ed941735',
      'name': 'Spa',
      'pluralName': 'Spas',
      'shortName': 'Spa',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594421783',
    'hasPerk': False},
   {'id': '4bc08ad7f8219c747aaeb110',
    'name': 'Beaches Rec Centre',
    'location': {'address': '6 Williamson Rd.',
     'crossStreet': 'at Lee Ave.',
     'lat': 43.67365887099423,
     'lng': -79.29863365009243,
     'l

In [128]:
prueba = results['response']['venues'][0]
prueba.keys()

dict_keys(['id', 'name', 'location', 'categories', 'referralId', 'hasPerk'])

Let´s clean the json and structure it into a pandas dataframe

In [129]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['venues']
    except:
        categories_list = row['categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Best Bathtub in the Beaches,Spa,43.674591,-79.293602
1,Beaches Rec Centre,College Gym,43.673659,-79.298634
2,Beaches Bake Shop,Bakery,43.680363,-79.289692
3,Beaches Rehabilitation Centre,Chiropractor,43.672440,-79.288636
4,Beaches Wholesome Market,Health Food Store,43.672661,-79.287567


In [130]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

34 venues were returned by Foursquare.


## Explore neighborhoods in Toronto City

Let's create a function to repeat the same process to all the neighborhoods in ECDW of Toronto

In [131]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_denc_venues

In [132]:
Toronto_ECDW_venues = getNearbyVenues(names=df_Toronto_ECDW['Neighborhood'],
                                   latitudes=df_Toronto_ECDW['Latitude'],
                                   longitudes=df_Toronto_ECDW['Longitude']
                                  )

In [133]:
Toronto_ECDW_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


Let's check how many venues were returned for each neighborhood

In [134]:
Toronto_ECDW_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,70,70,70,70,70,70
Christie,16,16,16,16,16,16
Church and Wellesley,71,71,71,71,71,71
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


Let's find out how many unique categories can be curated from all the returned venues

In [135]:
print('There are {} uniques categories.'.format(len(Toronto_ECDW_venues['Venue Category'].unique())))

There are 232 uniques categories.


## Analyze each Neighborhood

In [136]:
# one hot encoding
Toronto_ECDW_onehot = pd.get_dummies(Toronto_ECDW_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_ECDW_onehot['Neighborhood'] = Toronto_ECDW_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_ECDW_onehot.columns[-1]] + list(Toronto_ECDW_onehot.columns[:-1])
Toronto_ECDW_onehot = Toronto_ECDW_onehot[fixed_columns]

Toronto_ECDW_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [137]:
Toronto_ECDW_grouped = Toronto_ECDW_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_ECDW_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.214286,0.071429,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,Central Bay Street,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014286,0.0,0.0,0.014286,0.0,0.0


Check the 5 most common venues in each neighborhood

In [138]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_ECDW_grouped['Neighborhood']

for ind in np.arange(Toronto_ECDW_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_ECDW_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Bakery
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Brewery,Burrito Place,Spa
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Harbor / Marina,Boat or Ferry,Coffee Shop,Plane
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant


In [139]:
neighborhoods_venues_sorted.shape

(39, 6)

## Cluster neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [140]:
# set number of clusters
kclusters = 5

Toronto_ECDW_grouped_clustering = Toronto_ECDW_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_ECDW_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [158]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_ECDW_merged = df_Toronto_ECDW

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_ECDW_merged = Toronto_ECDW_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_ECDW_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Department Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Pub,Sandwich Place,Burrito Place,Board Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,Bakery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Women's Store,Dessert Shop


Finally, let's visualize the resulting clusters

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_denc_merged['Latitude'], 
        toronto_denc_merged['Longitude'], 
        toronto_denc_merged['Neighborhood'], 
        toronto_denc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster

### Cluster 1

In [155]:
Toronto_ECDW_merged.loc[Toronto_ECDW_merged['Cluster Labels'] == 0, Toronto_ECDW_merged.columns[[1] + list(range(5, Toronto_ECDW_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Women's Store,Department Store
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
2,East Toronto,0,Park,Pub,Sandwich Place,Burrito Place,Board Shop
3,East Toronto,0,Café,Coffee Shop,Brewery,Gastropub,Bakery
5,Central Toronto,0,Hotel,Department Store,Gym / Fitness Center,Park,Pizza Place
6,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Gym / Fitness Center,Fast Food Restaurant
7,Central Toronto,0,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Café
9,Central Toronto,0,Pub,Coffee Shop,Bagel Shop,Liquor Store,Restaurant
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Bakery,Pub
12,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant


### Cluster 2

In [156]:
Toronto_ECDW_merged.loc[Toronto_ECDW_merged['Cluster Labels'] == 1, Toronto_ECDW_merged.columns[[1] + list(range(5, Toronto_ECDW_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Central Toronto,1,Park,Lawyer,Trail,Restaurant,Deli / Bodega
10,Downtown Toronto,1,Park,Playground,Trail,Deli / Bodega,Event Space


### Cluster 3

In [152]:
Toronto_ECDW_merged.loc[Toronto_ECDW_merged['Cluster Labels'] == 2, Toronto_ECDW_merged.columns[[1] + list(range(5, Toronto_ECDW_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
23,Central Toronto,2,Trail,Mexican Restaurant,Jewelry Store,Sushi Restaurant,Women's Store


### Cluster 4

In [153]:
Toronto_ECDW_merged.loc[Toronto_ECDW_merged['Cluster Labels'] == 3, Toronto_ECDW_merged.columns[[1] + list(range(5, Toronto_ECDW_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
4,Central Toronto,3,Park,Bus Line,Swim School,Women's Store,Dessert Shop


### Cluster 5

In [154]:
Toronto_ECDW_merged.loc[Toronto_ECDW_merged['Cluster Labels'] == 4, Toronto_ECDW_merged.columns[[1] + list(range(5, Toronto_ECDW_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,4,Garden,Home Service,Department Store,Falafel Restaurant,Event Space
